# 머신러닝을 감성분석에 적용하기
* 텍스트 데이터의 정제 및 준비
* 텍스트 문서로부터 피처 벡터 생성하기
* 양과 음의 영화 리뷰를 분류하는 머신러닝 모델 학습하기
* 아웃 오브 코어(Out-of-core) 학습을 사용하여 방대한 텍스트 데이터 가지고 작업하기

## 1. IMDb 영화 리뷰 데이터 획득
영화 리뷰 데이터의 압축된 아카이브를 http://ai.stanford.edu/~amaas/data/sentiment/ 에서 gzip압축 tarball 아카이브로 내려받을 수 있다.

In [6]:
import pyprind
import pandas as pd
import os

pbar = pyprind.ProgBar(50000)
labels = {'pos':1, 'neg': 0}
df = pd.DataFrame()

for s in ('test','train'):
    for l in ('pos','neg'):
        path = './data/aclImdb/%s/%s' % (s, l)
        for file in os.listdir(path):
            with open(os.path.join(path,file),'r') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:05:12


numpy.random 하위 모듈의 permutation 함수를 사용해서 DataFrame을 셔플링한다. 셔플링한 영화 리뷰 데이터를 나중의 편의를 위해 CSV 파일로 저장한다.

In [7]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('./movie_data.csv', index=False)

In [8]:
df = pd.read_csv('./movie_data.csv')
df.head(3)

,review,sentiment
0,Good Film.<br /><br />I managed to pick this u...,1
1,After starting watching the re-runs of old Col...,0
2,"Silent Night, Deadly Night 5 is the very last ...",0


## 2. 백 오브 워드(bag-of-word) 모델
1. 고유 토큰(tokens) 단어집(vocabulary)을 만든다. 예, 전체 문서 집합의 단어들
2. 특정 문서에서 각각의 단어가 얼마나 자주 사용되었는지 횟수를 포함하는 각 문서에 대한 피처 벡터를 만든다.

### 단어를 피처 벡터로 변환
CountVectorizer 클래스는 텍스트 데이터를 배열로 취해서(문서가 될 수 있고 문장도 가능) 백 오브 워드 모델을 생성한다.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining and the weather is sweet'
])
bag = count.fit_transform(docs)

In [10]:
print(count.vocabulary_)

{'sweet': 4, 'sun': 3, 'shining': 2, 'the': 5, 'weather': 6, 'is': 1, 'and': 0}


In [11]:
print(bag.toarray())

[[0 1 1 1 0 1 0]
 [0 1 0 0 1 1 1]
 [1 2 1 1 1 2 1]]


###### n-gram 모델
'the sun is shining'에서 1-gram과 2-gram으로의 표현방식은 다음과 같이 구성될 것이다.
* 1-gram : "the", "sun", "is", "shining"
* 2-gram : "the sun", "sun is", "is shining"
사이킷런의 CountVectorizer 클래스를 이용하면 ngram_range 파라미터로 여러 n-gram 모델을 사용할 수 있다. 1-gram이 기본적으로 사용되지만 CountVectorizer 인스턴스를 새로 ngram_range=(2.2)로 설정하면 2-gram으로 바꿀 수 있다.

### 용어 빈도수 - 문서 빈도수의 역수를 이용한 단어의 관련성 평가
tf-idf(term frequency-inverse document frequency)는 피처 벡터에서 빈번하게 발생하는 단어들의 가중치를 낮추는 데 사용된다.

tf-idf는 용어 빈도수와 문서 빈도수의 역수의 곱으로 정의될 수 있다.

\begin{equation*}
\text{tf-idf(t,d)} = tf(t,d) \times idf(t,d) \\
idf(t,d) = \log\frac{n_d}{1+df(d,t)}
\end{equation*}

여기서 $n_d$는 문서의 전체 개수이며, $df(d,t)$는 용어 $t$를 포함하는 문서 $d$의 개수이다. 상수 $1$를 분모에 더하는 것은 학습 샘플 모두에 대해 분모가 $0$이 되지 않도록 만들어주는 역할을 한다. $\log$는 문서의 빈도가 낮은 경우 가중치가 너무 높아지지 않도록 해준다.

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[ 0.    0.43  0.56  0.56  0.    0.43  0.  ]
 [ 0.    0.43  0.    0.    0.56  0.43  0.56]
 [ 0.4   0.48  0.31  0.31  0.31  0.48  0.31]]


### 텍스트 데이터 정제

In [13]:
df.loc[2, 'review'][-50:]

' could get some of it out of my brain. 4 out of 5.'

In [14]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>','',text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-','')
    return text

preprocessor(df.loc[2, 'review'][-50:])

' could get some of it out of my brain 4 out of 5 '

DataFrame의 모든 영화 리뷰에 preprocessor 함수를 적용한다.

In [15]:
df['review'] = df['review'].apply(preprocessor)

### 문서를 토큰으로 처리하기

In [16]:
def tokenizer(text):
    return text.split()
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

NLTK 패키지의 포터 스테머를 사용하면 토큰화 함수를 수정해서 그 원형으로 단어를 축소할 수 있다. 단어 'running'은 그것의 원형인 'run'으로 스테밍된다.

In [17]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

stop-word란 모든 종류의 텍스트에서 공통으로 많이 사용되는 단어들로 문서의 다른 종류들을 구별하는 데 유용할 만한 정보를 거의 가지고 있지 않은(혹은 아주 조금만 가지고 있는) 경우를 말한다. 예, is, and, has 같은 것들이 있다.

In [18]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/yjh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

## 3. 문서 분류를 위한 로지스틱 회귀 모델 훈련

우선, 정제된 텍스트 문서의 DataFrame을 문서 2만5천 개는 훈련용으로 문서 2만5천 개는 테스트용으로 나눌 것이다.

In [20]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

이제 GridSearchCV 오브젝트를 사용하여 5-fold 층화 교차검증을 사용하는 이번 로지스틱 회귀 모델에 대한 최적의 파라미터 세트를 찾아보자.

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)
param_grid = [{'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1','l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range':[(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf': [False],
               'vect__norm': [None],
               'clf__penalty': ['l1','l2'],
               'clf__C': [1.0, 10.0, 100.0]}
             ]
lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy',cv=5, verbose=1,n_jobs=-1)
#gs_lr_tfidf.fit(X_train, y_train) # 40분이상 소요...

/home/yjh/learn/python-machine-learning/venv/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/yjh/learn/python-machine-learning/venv/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 36.6min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 166.9min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 211.0min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid=[{'clf__C': [1.0, 10.0, 100.0], 'vect__stop_words': [['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 't...e_idf': [False], 'vect__norm': [None], 'vect__ngram_range': [(1, 1)], 'clf__C': [1.0, 10.0, 100.0]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=1

## 4. 더 큰 데이터로 작업하기 - 온라인 알고리즘과 아웃 오브 코어 학습